# Credmark Modeling Framework Example for Jupyter notebook

## Ledger Model

version: 2023.4.15


## Initialize

<div class="alert alert-block alert-info">
    <b>Note:</b> Change to a web3 provider you have in `params`/`chain_to_provider_url` below.
</div>


In [4]:
%reload_ext credmark.cmf.ipython

cmf_param = {
    'chain_id': 1,
    'block_number': None,
    # 'chain_to_provider_url': {'1': 'https://mainnet.infura.io/v3/... or https://eth-mainnet.g.alchemy.com/'}, # or, use the credmark's nodes
    'api_url': None,
    'use_local_models': None,
    'register_utility_global': True}

context, model_loader = %cmf cmf_param

## Example


### Contract - Uni V2


In [5]:
contract = Contract(address='0x0d4a11d5EEaaC28EC3F61d100daF4d40471f1852')
print(contract.functions.name().call(), contract.functions.symbol().call())

for addr in [contract.functions.token0().call(), contract.functions.token1().call()]:
    print(Token(address=addr).symbol)

Uniswap V2 UNI-V2
WETH
USDT


### Ledger Examples


- Get the max value group by `to`


In [6]:
contract = Contract(address='0x0d4a11d5EEaaC28EC3F61d100daF4d40471f1852')

with contract.ledger.functions.Transfer as q:
    ret = (q.select(
        aggregates=[(q.FN_VALUE.max_(), 'max_value')],
        group_by=[q.FN_TO,q.BLOCK_NUMBER],
        order_by=q.field('max_value').dquote().desc(),
        where=q.BLOCK_NUMBER.gt(context.block_number - 2000),
        limit=10
    ).to_dataframe())
display(ret)

""


- Get max amount0Out by `to`.


In [7]:
with contract.ledger.events.Swap as q:
    ret = (q.select(
        aggregates=[(q.EVT_AMOUNT0OUT.max_(), 'max_value')],
        group_by=[q.EVT_TO],
        order_by=q.field('max_value').dquote().desc(),
        where=q.BLOCK_NUMBER.gt(context.block_number - 2000000),
        limit=5)
    .to_dataframe())
display(ret)

,max_value,evt_to
0,999998367510990867,0x1111111254eeb25477b68fb85ed929f73a960582
1,999998231482191861,0x1111111254fb6c44bac0bed2854e76f90643097d
2,999998035187130322,0x6033368e4a402605294c91cf5c03d72bd96e7d8d
3,999995117313840887,0x0a361804715edcfc61bce38bead4b5e4a49ec921
4,99999059885198703,0x554f01d148d719a888a35c991f367ff2771b1a02


- Get max amount0Out and max amount1Out


In [8]:
contract = Contract(address='0x0d4a11d5EEaaC28EC3F61d100daF4d40471f1852')
with contract.ledger.events.Swap as q:
    ret = (q.select(
        aggregates=[(q.EVT_AMOUNT0OUT.max_(), 'max_amount0Out'),
                    (q.EVT_AMOUNT1OUT.max_(), 'max_amount1Out')]
    ).to_dataframe())
display(ret)

,max_amount0Out,max_amount1Out
0,99999999980354914,99999999999


- Get the transaction hash containing the max amount0Out.


In [9]:
contract = Contract(address='0x0d4a11d5EEaaC28EC3F61d100daF4d40471f1852')
with contract.ledger.events.Swap as q:
    ret2 = (q.select(
        aggregates=[(q.EVT_AMOUNT0OUT.max_(), 'max_amount0Out')],
        group_by=[q.TXN_HASH],
        having=q.EVT_AMOUNT0OUT.max_().ge(ret.max_amount0Out[0]),
    ).to_dataframe())
display(ret2)

,transaction_hash,max_amount0Out
0,0x8f2fe3f7de818bb86552b0a8973fece583f33c4960ff...,99999999980354914


- Get the transaction hash containing the max amount0Out or max amount1Out

For any column in `having` needs to be in `aggregates`.


In [10]:
contract = Contract(address='0x0d4a11d5EEaaC28EC3F61d100daF4d40471f1852')
with contract.ledger.events.Swap as q:
    ret3 = (q.select(
        aggregates=[(q.EVT_AMOUNT0OUT.max_(), 'max_amount0Out'),
                    (q.EVT_AMOUNT1OUT.max_(), 'max_amount1Out')],
        group_by=[q.BLOCK_NUMBER, q.TXN_HASH],
        having=(
            q.EVT_AMOUNT0OUT.max_().ge(ret.max_amount0Out[0]).or_(
            q.EVT_AMOUNT1OUT.max_().ge(ret.max_amount1Out[0]))),
        order_by=q.BLOCK_NUMBER
    ).to_dataframe())

display(ret3)

,block_number,transaction_hash,max_amount0Out,max_amount1Out
0,12381561,0x12c145ecce24b060c71a28ebebb6ae65285cdc06a19a...,0,99999999999
1,12387763,0x533c7163f91a67b0ed01d68510592bed7bbe3272c04e...,0,99999999999
2,12388027,0xb75a60a69afdc7bf35f3adab6b641580886110c32de4...,0,99999999999
3,12390722,0xbdd434496b5ceb6aa6a859ae6c06359dcfaed0096156...,0,99999999999
4,12392318,0x04884fd5356b60049a823876114a531a994dc119aae4...,0,99999999999
5,12392343,0x046cf2ae940ab71d66df752d491ce13fc19f5d251336...,0,99999999999
6,12392388,0x9168801ba0d802bc48e514fc0231c656a2873c7e8a65...,0,99999999999
7,12392413,0x7ba688f3ee9a141f83300f92c3b7d097952104b505bc...,0,99999999999
8,12392448,0x3d1dd49ca7ebda02b2e94a71affcc4d4307e3948327a...,0,99999999999
9,12393596,0x2bf8a4475d94de809d0322e87ce260db38db4832111e...,0,99999999999


In [11]:
# Below web3 may not work with some node RPC providers
if False:
    contract = Contract(address='0x0d4a11d5EEaaC28EC3F61d100daF4d40471f1852')
    swap_events = contract.events.Swap.createFilter(
        fromBlock=0,
        toBlock=context.block_number
    ).get_all_entries()


In [12]:
10 / 100, block_number - 1000, block_number - 990

(0.1, 17475070, 17475080)

In [13]:
contract = Contract(address='0x0d4a11d5EEaaC28EC3F61d100daF4d40471f1852')
pd.DataFrame(contract.fetch_events(contract.events.Swap, from_block=block_number-1000, to_block=block_number-990, by_range=11))

,args,event,logIndex,transactionIndex,transactionHash,address,blockHash,blockNumber,sender,to,amount0In,amount1In,amount0Out,amount1Out
0,"(sender, to, amount0In, amount1In, amount0Out,...",Swap,131,52,b'*\x14\xc2\xff\xaa\xbd\xdb4\xe1\x07\xea\x0e$K...,0x0d4a11d5EEaaC28EC3F61d100daF4d40471f1852,b'.r\x91NUw\xac\xe0\x8cFS\x85v.n\x1c\xad\x9c\x...,17475074,0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D,0x44943241F6491d5f3FD833788AdE6941967Cb9e2,950000000000000000,0,0,1654746914
1,"(sender, to, amount0In, amount1In, amount0Out,...",Swap,118,61,b'\x97\xe6\xbb\x17w\xb2\xe8\xaac5j9~\xa6\xbc\x...,0x0d4a11d5EEaaC28EC3F61d100daF4d40471f1852,b'\x1e&~\na\xcbv\xe6}l5\t\xf8\x13gNT\nI\x8d\xe...,17475078,0x3fC91A3afd70395Cd496C647d5a6CC9D4B2b7FAD,0x1A5B87FF2e576B0A4605206c262A3D793BaB725a,467841993800675236,0,0,814837411
